# Run MethaneMIP scenarios

Use fair calibration v1.4.0, which is appropriate for SSP scenarios with corrected NOx emissions.

In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
f = FAIR(ch4_method="Thornhill2021")

MethaneMIP scenarios are defined to 2050. CMIP7 might run to 2125, so I have extended to 2030 since emissions are given at 10 years

In [ ]:
f.define_time(1750, 2131, 1)

In [ ]:
scenarios = [
    'ssp245',
    'ssp245-ModAggr',
    'ssp245-Aggr',
]

In [ ]:
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_4_0_file = '../data/parameters/calibrated_constrained_parameters_1.4.0.csv'

In [ ]:
df_configs = pd.read_csv(fair_params_1_4_0_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
fair_species_configs_1_4_0_file = '../data/parameters/species_configs_properties_1.4.0.csv'

In [ ]:
species, properties = read_properties(filename=fair_species_configs_1_4_0_file)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_csv(
    emissions_file='../data/emissions/methane-emissions-scenarios.csv',
    forcing_file='../data/forcing/volcanic_solar.csv',
)

In [ ]:
fill(
    f.forcing,
    f.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)

In [ ]:
f.fill_species_configs(fair_species_configs_1_4_0_file)
f.override_defaults(fair_params_1_4_0_file)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.concentration.sel(specie='CH4', scenario='ssp245'), color='r');
pl.plot(f.concentration.sel(specie='CH4', scenario='ssp245-ModAggr'), color='k');
pl.plot(f.concentration.sel(specie='CH4', scenario='ssp245-Aggr'), color='b');
pl.xlim(250, 300)

In [ ]:
pl.plot(10 * f.alpha_lifetime.sel(specie='CH4', scenario='ssp245'), color='r');
pl.plot(10 * f.alpha_lifetime.sel(specie='CH4', scenario='ssp245-ModAggr'), color='k');
pl.plot(10 * f.alpha_lifetime.sel(specie='CH4', scenario='ssp245-Aggr'), color='b');
pl.xlim(250, 300)

In [ ]:
pl.plot(f.temperature.sel(layer=0, scenario='ssp245').median(dim='config'), color='r');
pl.plot(f.temperature.sel(layer=0, scenario='ssp245-ModAggr').median(dim='config'), color='k');
pl.plot(f.temperature.sel(layer=0, scenario='ssp245-Aggr').median(dim='config'), color='b');
pl.xlim(250, 300)
pl.ylim(0.5, 2.0)

In [ ]:
f.forcing.sel(specie='CH4', scenario='ssp245') - f.forcing.sel(specie='CH4', scenario='ssp245-Aggr')
f.forcing.sel(specie='Ozone', scenario='ssp245') - f.forcing.sel(specie='Ozone', scenario='ssp245-Aggr')
f.forcing.sel(specie='Stratospheric water vapour', scenario='ssp245') - f.forcing.sel(specie='Stratospheric water vapour', scenario='ssp245-Aggr')
f.forcing.sel(specie='Aerosol-radiation interactions', scenario='ssp245') - f.forcing.sel(specie='Aerosol-radiation interactions', scenario='ssp245-Aggr')

#f.forcing.sel(specie='Ozone')
#f.forcing.sel(specie='Aerosol-radiation interactions')

In [ ]:
pd.DataFrame(
    f.concentration.sel(specie='CH4').interp(timebounds=f.timepoints).median(dim='config'),
    index=f.timepoints,
    columns=f.scenarios
).to_csv('../output/ch4-concentration-medians.csv')